In [1]:
# this code is to convert the outdated gene name in gene.covariates.txt into HGNC symbol, the table is generated from 
# an R convert code using Biomart
# Author: Yiyun Rao

import pandas as pd
import re, os, pickle

In [14]:
dir_ref = '../anno_ref/proc_refs'
dir_mutsig_in = '../anno_ref/MutSigCVsyn_inputs'

lgene = pickle.load(open(os.path.join(dir_ref,'gene_name_list_062121.pkl'),'rb'))
df_covar = pd.read_csv(os.path.join(dir_mutsig_in, 'gene.covariates.txt'),sep = '\t',index_col = 0)

***
**Old code***

In [2]:
#pcawg all gene list(from all unique genes in coverage file)
pcawg_f = '../anno_ref/pcawg_genelist.txt'
pcawg = open(pcawg_f,'r')
list_pcawg= []
for lines in pcawg:
    genes = lines.strip('\n')
    list_pcawg.append(genes)

#load file
covar_f = '../anno_ref/gene.covariates.txt'
covar = open(covar_f,'r')
next(covar)
list_covar =[]
for lines in covar:
    gene = lines.split('\t')[0]
    list_covar.append(gene)

In [3]:
# open table and read as dictionary
tab = "../anno_ref/gene_synonym_table.txt"
dict_syn = {}
with open(tab, 'r') as tabf:
    next(tabf)
    for lines in tabf:
        line_split = lines.strip('\n').split('\t')
        gene_hgnc = line_split[0]
        gene_synlist = line_split[1:]

        for gsyn in gene_synlist:
            gsyn = gsyn.strip(' ')
            if gsyn not in dict_syn:
                dict_syn[gsyn] = gene_hgnc 

In [4]:
# (external)open table and read as dictionary
tab = "../anno_ref/gene.covariates.external.txt"
dict_syn = {}
with open(tab, 'r') as tabf:
    next(tabf)
    for lines in tabf:
        line_split = lines.strip('\n').split('\t')
        gene_hgnc = line_split[0]
        gene_syn = line_split[1]
        if gene_syn not in dict_syn:
            dict_syn[gene_syn] = gene_hgnc

In [5]:
# read covariates data as pandas dataframe
covar_f = '../anno_ref/gene.covariates.txt'
df_cov = pd.read_csv(covar_f, sep = '\t')

In [6]:
# open covariate file and find gene synom
dict_newname = {}
# cov_m = open('../anno_ref/gene.covariates.converted.txt' ,'wb')
list_cgene = []
with open(covar_f, 'r') as covar:
    for lines in covar:
        gene = lines.split('\t')[0]
        newname = gene
        try:
            newname = dict_syn[gene]
            if newname not in list_covar and newname not in list_cgene: # already exist and duplicate genes not updated
                df_cov.replace(to_replace=r'^'+re.escape(gene)+'$', value=newname, regex=True, inplace = True)
        except KeyError:
            pass
        list_cgene.append(newname)

In [10]:
# write df_cov to pandas dataframe
df_cov.to_csv('../anno_ref/gene.covariates.converted.txt', index=False, header= True, sep='\t')

In [22]:
df_cov[df_cov['gene'].duplicated()]

,gene,expr,reptime,hic


In [23]:
len(list_cgene)

18268

In [24]:
# genes in new covariates not in old covariates
list_gc = []
for g in list_cgene:
    if g not in list_covar:
        list_gc.append(g)
print len(list_gc)

816


In [25]:
# gene in new covariates not in pcawg
list_pnnc = []
for g in list_cgene:
    if g not in list_pcawg:
        list_pnnc.append(g)
print len(list_pnnc)

2631


In [26]:
# gene in pcawg not in new covariates
list_cnnp = []
for g in list_pcawg:
    if g not in list_cgene:
        list_cnnp.append(g)
print len(list_cnnp)

1863
